In [1]:
import numpy as np
import pandas as pd
import haversine
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
fn_out = "../output/nn5_BASS_ALBAY.csv"

In [6]:
fn = "../data/BASSCellSiteLocation.csv"
df = pd.read_csv(fn)

In [7]:
len(df)

33673

In [8]:
df.head(2)

,CID,LAC,Operator,Province,Municipality,GPSCoordinates,MedianBandwidth,Longitude,Latitude,LEN
0,35540737.0,3004,NaN,Cavite,General Mariano Alvarez,"121.01253750000001,14.302141000000001",28.0,121.012537,14.302141,19
1,223188122.0,35601,NaN,Sorsogon,Matnog,"124.09332000000001,12.53529",43.0,124.093320,12.535290,19


In [9]:
df1 = df[df["Province"]=="Albay"]
len(df1)

91

In [10]:
df_sub = df1.copy().reset_index()
nsub = len(df_sub)

lon = df_sub["Longitude"]
lat = df_sub["Latitude"]

# initialize output dataframe
df_out = pd.DataFrame()
cols = ["ID", "JUNK", "LONGITUDE", "LATITUDE", "ID1", "ID2", "ID3", "ID4", "ID5", "D1", "D2", "D3", "D4", "D5"]
for col in cols:
    df_out[col] = np.zeros(nsub)

for i in np.arange(nsub):
    school_id = i #df_sub.loc[i, "SCHOOL ID"]
    source_coord = "" #df_sub.loc[i, "SOURCE_COORD"]
    di = np.zeros(nsub) + np.nan # initialize array for distances to other schools
    lon1, lat1 = lon[i], lat[i]
    for j in np.arange(nsub):    
        lon2, lat2 = lon[j], lat[j]
        coord1, coord2 = (lon1, lat1), (lon2, lat2)
        dij = haversine.Haversine(coord1, coord2).km
        di[j] = dij
    # get nearest nn neighbors
    jsort = np.argsort(di)
    jnn = jsort[1:6]  # remove i=j
    idnn = jnn #df_sub.loc[jnn, "SCHOOL ID"].values
    dnn = di[jnn]
    out = np.array([school_id, source_coord, lon1, lat1, idnn[0], idnn[1], idnn[2], idnn[3], idnn[4], dnn[0], dnn[1], dnn[2], dnn[3], dnn[4]])
    df_out.loc[i,:] = out

In [13]:
df_out.to_csv(fn_out, index=False)